In [228]:
#| default_exp apis
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# APIS

> Este módulo concentra funções as constantes, funções de carga, processamento e salvamento de dados aeronáuticos externos à ANATEL. Provenientes principalmente de APIs, como o AISWEB e REDEMET, mas também outras fontes como o Software Frequency Finder da ICAO. 

In [229]:
#| export
import json
from datetime import datetime
from urllib.request import urlopen
import xml.etree.ElementTree as ET
from typing import Union, Iterable

import requests
import xmltodict
import pandas as pd

## CONSTANTES


Dados para acesso à API GEOAISWEB

In [230]:
#| export
LINK_VOR = f"https://geoaisweb.decea.mil.br/geoserver/ICA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ICA:vor&outputFormat=application%2Fjson"
LINK_DME = f"https://geoaisweb.decea.mil.br/geoserver/ICA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ICA:dme&outputFormat=application%2Fjson"
LINK_NDB = f"https://geoaisweb.decea.mil.br/geoserver/ICA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ICA:ndb&outputFormat=application%2Fjson"
COLS_VOR = (
    "properties.frequency",
    "properties.frequnits",
    "properties.latitude",
    "properties.longitude",
    "properties.tipo",
    "properties.txtname",
    "properties.txtrmk",
)
COLS_NDB = (
    "properties.valfreq",
    "properties.uomfreq",
    "properties.geolat",
    "properties.geolong",
    "properties.tipo",
    "properties.txtname",
    "properties.txtrmk",
)

COLS_DME = (
    "properties.valchannel",
    "properties.codechanne",
    "properties.geolat",
    "properties.geolong",
    "properties.tipo",
    "properties.txtname",
    "Channel",
)

# PATH_CHANNELS = r"\\servrepds\dw$\Input\sentinela\extracao\VOR_ILS_DME_Channel.xlsx"
PATH_CHANNELS = Path.cwd().parent.parent / "dados" / "VOR_ILS_DME_Channel.xlsx"
DF_CHANNELS = pd.read_excel(PATH_CHANNELS, engine="openpyxl", dtype="string")

In [231]:
#| export
def convert_frequency(
    freq: float,  # Frequência Central da emissão
    unit: str,  # Unidade da Frequência: [Hz, kHz, MHZ, GHZ]
) -> float:  # Frequência em MHz
    """Converte a frequência `freq` para MHz"""
    match unit.upper():
        case "HZ":
            result = freq / 1e6
        case "KHZ":
            result = freq / 1000
        case "MHZ":
            result = freq
        case "GHZ":
            result = freq * 1000
        case _:
            result = -1
    return result


In [232]:
#| export
def _process_frequency(
    df: pd.DataFrame,  # Dataframe com os dados
    cols: Iterable[str],  # Subconjunto de Colunas relevantes do DataFrame
) -> pd.DataFrame:  # Dataframe com os dados de frequência devidamente processados
    if cols == COLS_DME:
        df.dropna(subset=[cols[0]], inplace=True)
        df["Channel"] = df[cols[0]].astype("int").astype("string") + df[cols[1]]
        df["frequency"] = -1.0

        for row in df.itertuples(index=True):
            row_match = DF_CHANNELS.loc[
                (DF_CHANNELS.Channel == row.Channel), "DMEground"
            ]
            if not row_match.empty:
                df.loc[row.Index, "frequency"] = float(row_match.item())

    else:
        df["frequency"] = (
            df[[cols[0], cols[1]]]
            .apply(lambda x: convert_frequency(x[0], x[1]), axis=1)
            .astype("float")
        )
    return df


In [233]:
#| export
def _filter_df(df, cols):  # sourcery skip: use-fstring-for-concatenation
    df["Description"] = (
        "[AISG] " + df[cols[4]] + " - " + df[cols[5]] + " " + df[cols[6]]
    ).astype("string")

    df = df[["frequency", cols[2], cols[3], "Description"]]

    return df.rename(
        columns={
            cols[2]: "latitude",
            cols[3]: "longitude",
        }
    )

In [234]:
#|export
def get_geodf(
    link: str,  # Link para a requisição das estações VOR do GEOAISWEB
    cols: Iterable[str],  # Subconjunto de Colunas relevantes do DataFrame
) -> pd.DataFrame:  # DataFrame com frequências, coordenadas e descrição das estações VOR
    # sourcery skip: use-fstring-for-concatenation
    """Faz a requisição do `link`, processa o json e o retorna como Dataframe"""
    response = urlopen(link)
    if (
        response.status != 200
        or "application/json" not in response.headers["content-type"]
    ):
        raise ValueError(
            f"Resposta a requisição não foi bem sucedida: {response.status=}"
        )
    data_json = json.loads(response.read())
    df = pd.json_normalize(
        data_json["features"],
    ).filter(cols, axis=1)
    df = _process_frequency(df, cols)
    return _filter_df(df, cols)


In [235]:
get_geodf(LINK_VOR, COLS_VOR)

,frequency,latitude,longitude,Description
0,116.8,-16.245367,-48.979089,[AISG] VOR - ANÁPOLIS CH 101X
1,117.0,-19.689048,-47.060575,<NA>
2,113.4,-9.868361,-56.104951,[AISG] VOR - ALTA FLORESTA CH 81X OPR INFRAERO
3,116.2,-22.951451,-46.569805,[AISG] VOR - BRAGANÇA CH 109X
4,114.3,-12.079940,-45.007135,[AISG] VOR - BARREIRAS CH 90X
...,...,...,...,...
74,112.1,-25.583203,-54.503514,[AISG] VOR - FOZ CH 58X
75,115.3,-31.390714,-54.109717,[AISG] VOR - BAGÉ U/S BTN RDL 275/305
76,116.9,-23.627464,-46.654635,[AISG] VOR - CONGONHAS CH 116X \nVOR/DME NO AV...
77,115.9,-14.799000,-64.938333,[AISG] VOR - TRINIDAD-BL See Bolivia AIP


In [236]:
get_geodf(LINK_NDB, COLS_NDB)

,frequency,latitude,longitude,Description
0,0.2650,-21.139333,-50.426667,[AISG] NDB - ARAÇATUBA OPR TAM
1,0.3000,-25.402667,-49.229000,"[AISG] NDB - BACACHERI FM ARP, COVERAGE 50NM"
2,0.3800,-22.314000,-49.107167,[AISG] NDB - BAURU COVERAGE 50NM OPR INFRAERO
3,0.2300,-7.266000,-35.892667,[AISG] NDB - CAMPINA GRANDE COVERAGE 60NM OPR ...
4,0.4200,-20.813167,-49.406167,[AISG] NDB - RIO PRETO OPR DAESP
5,0.4070,-4.194997,-69.939733,[AISG] NDB - LETÍCIA OPR COLÔMBIA
6,0.2500,-29.694722,-57.148056,[AISG] NDB - PASO DE LOS LIBRES OPR ARGENTINA
7,0.2950,-19.658900,-43.896933,[AISG] NDB - LAGOA SANTA COVERAGE 50NM
8,0.3750,-19.016219,-57.664456,<NA>
9,0.2050,-5.386167,-35.531000,<NA>


In [237]:
get_geodf(LINK_DME, COLS_DME)

,frequency,latitude,longitude,Description
0,1019.0,-22.812774,-42.095339,[AISG] DME - ALDEIA 58X
1,1202.0,-16.245367,-48.979089,[AISG] DME - ANÁPOLIS 115X
2,1204.0,-19.689048,-47.060575,[AISG] DME - ARAXÁ 117X
3,1168.0,-9.868361,-56.104951,[AISG] DME - ALTA FLORESTA 81X
4,1196.0,-22.951358,-46.569900,[AISG] DME - BRAGANÇA 109X
...,...,...,...,...
148,1175.0,-25.537761,-48.529855,[AISG] DME - PARANAGUÁ 88X
149,1181.0,-18.203235,-45.457072,[AISG] DME - TRÊS MARIAS 94X
150,1171.0,-27.621855,-48.632464,[AISG] DME - BIGUAÇU 84X
151,1186.0,-14.907781,-40.918839,[AISG] DME - VITÓRIA DA CONQUISTA 99X


In [238]:
#| export
def get_aisg_data() -> pd.DataFrame:  # DataFrame com todos os dados do GEOAISWEB
    """Lê e processa os dataframes individuais da API GEOAISWEB e retorna o conjunto concatenado"""
    return pd.concat(
        get_geodf(link, cols)
        for link, cols in zip(
            [LINK_NDB, LINK_VOR, LINK_DME], [COLS_NDB, COLS_VOR, COLS_DME]
        )
    )

In [239]:
get_aisg_data()

,frequency,latitude,longitude,Description
0,0.265,-21.139333,-50.426667,[AISG] NDB - ARAÇATUBA OPR TAM
1,0.300,-25.402667,-49.229000,"[AISG] NDB - BACACHERI FM ARP, COVERAGE 50NM"
2,0.380,-22.314000,-49.107167,[AISG] NDB - BAURU COVERAGE 50NM OPR INFRAERO
3,0.230,-7.266000,-35.892667,[AISG] NDB - CAMPINA GRANDE COVERAGE 60NM OPR ...
4,0.420,-20.813167,-49.406167,[AISG] NDB - RIO PRETO OPR DAESP
...,...,...,...,...
148,1175.000,-25.537761,-48.529855,[AISG] DME - PARANAGUÁ 88X
149,1181.000,-18.203235,-45.457072,[AISG] DME - TRÊS MARIAS 94X
150,1171.000,-27.621855,-48.632464,[AISG] DME - BIGUAÇU 84X
151,1186.000,-14.907781,-40.918839,[AISG] DME - VITÓRIA DA CONQUISTA 99X
